In [1]:
from math import log
import numpy as np

In [4]:
# 数据读取并处理
label=[]
feature=np.zeros([32561,124])
f=open('Tree.txt')
line = f.readline()
a=0
while line:
    data=[]
    for i in line.split():
        data.append(i);
    for i in data[1:]:
        j=i.split(":")
        feature[a][int(j[0])-1]=int(j[1])
    if data[0] in '+1':
        feature[a][123]=1
    else:
        feature[a][123]=0
    line=f.readline()
    a+=1
f.close

#构建训练集和测试集
feature1 = feature[5000:32561]
feature = feature[0:5000]

In [5]:
# 统计每个特征中存在的取值
def classnums(data):
    cla=[]
    for i in data:
        if i not in cla:
            cla.append(i)
            
    return cla

In [6]:
# 计算经验熵
def calentropy(data):
    names=locals()
    n=len(data)
    p=[]
    H=0
    cla=classnums(data)
    for i in cla:
        names['class%s'%i]=data.count(i)
        p.append(data.count(i)/n)
    for i in p:
        H -=i*log(i,2)
    return H

#计算条件经验熵
def calentropy2(data1,data2):
    h=0
    n=len(data1)
    names=locals()
    dic={}
    cla=classnums(data1)
    cla1=classnums(data2)
    for i in cla1:
        names['class%s'%i]=[]
    for i in range(n):
        a=data1[i]
        b=data2[i]
        dic[b]=names['class%s'%b]
        names['class%s'%b].append(a)
    for i in dic:
        H=calentropy(dic[i])
        m=len(dic[i])
        H=(m/n)*H
        h=h+H
    return h

#切割数据集
def spiltdata(data,axis,value):
    newdata=[]
    for i in data:
        if i[axis]==value:
            onedata=np.delete(i,axis,axis=0)
            newdata.append(onedata)
    newdata=np.array(newdata,dtype=float)
    return newdata

#计算信息增益或者信息增益比
def chooseBestFeature(feature,c):
    feature_nums=len(feature[0])-1
    label=feature[:,-1]
    label=label.tolist()
    feature=feature[:,0:feature_nums]
    h=calentropy(label)
    hbest=0
    global feature_idx
    for i in range(feature_nums):
        a = feature[:,i]
        H = calentropy2(label,a)
        if c == 'id3':
            h1 = h - H
        if c == 'c4.5':
            h1 = (h-H)/h
        if hbest < h1:
            hbest = h1
            feature_idx = i
    return feature_idx,hbest

#占比最大的类作为标记
def maxkey(feature):
    cla = classnums(feature)
    max1 = 0
    for i in cla:
        feature1 = feature.tolist()
        a = feature1.count(i)
        if a > max1:
            max1 = a
            maxkey = i
    return maxkey

#生成树
def createtree(feature,thre,c):
    if len(feature[0]) == 1:
        return maxkey(feature)
    label = feature[:,-1]
    cla = classnums(label)
    if len(cla) == 1:
        return cla[0]
    bestfeature,hbest = chooseBestFeature(feature,c)
    if hbest < thre:
        return maxkey(feature[:,bestfeature])
    tree = {bestfeature:{}}
    bestfeatureclass = classnums(feature[:,bestfeature])
    for i in bestfeatureclass:
        tree[bestfeature][i] = createtree(spiltdata(feature,bestfeature,i),thre,c)
    return tree

#分类
def classify(data,tree):
    a = list(tree.keys())[0]
    lasta = tree[a]
    key = data[a]
    vof = lasta[key]
    if type(vof) == dict:
        classlabel = classify(data,vof)
    else:
        classlabel = vof
    return classlabel

In [7]:
#计算准确率
def acc(feature,a):
    n = len(feature)
    t = 0
    for i in range(n):
        data=feature[i,0:122]
        b=classify(data,a)
        c=feature[i,-1]
        if b==c:
            t+=1
    return t/n

Tree=createtree(feature,0.04,'id3')
print(Tree)
print(acc(feature,Tree))
print(acc(feature,Tree))

{39: {0.0: {38: {1.0: {71: {0.0: {52: {1.0: 0.0, 0.0: {58: {0.0: {48: {0.0: {13: {1.0: 1.0, 0.0: {13: {0.0: {29: {0.0: {4: {0.0: {35: {0.0: {70: {0.0: {5: {1.0: 0.0, 0.0: {2: {0.0: {3: {0.0: {12: {1.0: 1.0, 0.0: 0.0}}, 1.0: 0.0}}, 1.0: 1.0}}}}, 1.0: 1.0}}, 1.0: {0: {0.0: 0.0, 1.0: {11: {1.0: 1.0, 0.0: 0.0}}}}}}, 1.0: 0.0}}, 1.0: 1.0}}, 1.0: {36: {0.0: 1.0, 1.0: 0.0}}}}}}, 1.0: {1: {0.0: 1.0, 1.0: {37: {1.0: 0.0, 0.0: 1.0}}}}}}, 1.0: 0.0}}}}, 1.0: {0: {1.0: 0.0, 0.0: 0.0}}}}, 0.0: 0.0}}, 1.0: {38: {1.0: 1.0, 0.0: 0.0}}}}
0.8028
0.8028
